# Use examples of [edges](https://github.com/romainsacchi/edges)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `edge` to use exchange-specific
characterization factors in the characterization matrix of `bw2calc`, combining the use of exchange names and locations.

## Requirements

* **Pyhton 3.10 or higher (up to 3.11) is highly recommended**

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have an activated `brightway2` project with a `biosphere3` database as well as an [ecoinvent](https://ecoinvent.prg) v.3 cut-off or consequential database registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.

In [1]:
from edges import EdgeLCIA, get_available_methods
import bw2data

One can simply build its own LCIA file.
Let's consider the following LCIA file (saved under `lcia_example_2.json`):

In [2]:
[
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process",
      "location": "CH" # now, we want to specify the location of the consumer
    },
    "value": 1.0
  },
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process",
      "location": "FR" # now, we want to specify the location of the consumer
    },
    "value": 2.0
  },
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process",
      "location": "RER" # now, we want to specify the location of the consumer
    },
    "value": 3.0
  }
]

[{'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process', 'location': 'CH'},
  'value': 1.0},
 {'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process', 'location': 'FR'},
  'value': 2.0},
 {'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process', 'location': 'RER'},
  'value': 3.0}]

In [2]:
# activate the bw project
bw2data.projects.set_current("ecoinvent-3.10-cutoff")
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
act

'sawing, softwood' (kilogram, CA-QC, None)

In [25]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=("some", "method"),
    filepath="lcia_example_2.json"
)
LCA.lci()

LCA.map_exchanges()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|██████████| 3/3 [00:00<00:00, 268.46it/s]


0.0009140300372816784

### Generate dataframe of characterization factors used

The `generate_cf_table` method generates a dataframe of the characterization factors used in the calculation. One can see the characterization factors used for each exchange in the system.

In [26]:
df = LCA.generate_cf_table()

In [27]:
# we can see under the "CF" column
# the characterization factors used for each exchange in the system
df

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Carbon dioxide, non-fossil","(air, urban air close to ground)","treatment of waste wood, untreated, municipal ...","waste wood, untreated",CH,5.599001e-06,1.0,5.599001e-06
1,"Carbon dioxide, non-fossil","(air, urban air close to ground)","biomethane, low pressure burned in micro gas t...","heat, central or small-scale, other than natur...",CH,5.222896e-20,1.0,5.222896e-20
2,"Carbon dioxide, non-fossil","(air, urban air close to ground)","treatment of digester sludge, municipal incine...","heat, for reuse in municipal waste incineratio...",CH,8.458534e-09,1.0,8.458534e-09
3,"Carbon dioxide, non-fossil","(air, urban air close to ground)","lath, hardwood, raw, kiln drying to u=20%","sawnwood, lath, hardwood, raw, dried (u=20%)",CH,3.682106e-10,1.0,3.682106e-10
4,"Carbon dioxide, non-fossil","(air, urban air close to ground)",beet sugar production,sugar beet pulp,CH,2.964306e-29,1.0,2.964306e-29
...,...,...,...,...,...,...,...,...
1205,"Carbon dioxide, to soil or biomass stock","(soil, agricultural)",esters of versatic acid production,esters of versatic acid,RER,5.126238e-16,3.0,1.537871e-15
1206,"Carbon dioxide, to soil or biomass stock","(soil, agricultural)",citric acid production,"protein feed, 100% crude",RER,4.989133e-39,3.0,1.496740e-38
1207,"Carbon dioxide, to soil or biomass stock","(soil, agricultural)",anthraquinone production,anthraquinone,RER,1.552314e-14,3.0,4.656941e-14
1208,"Carbon dioxide, to soil or biomass stock","(soil, agricultural)",latex production,latex,RER,6.797832e-19,3.0,2.039350e-18


As expected, only CH, FR and RER-based consumers have been considered in the calculation.

In [28]:
df.groupby("consumer location")["CF"].mean()

consumer location
CH     1.0
FR     2.0
RER    3.0
Name: CF, dtype: float64

But `statistics()` tells us that many exchanges have been ignored.

In [29]:
LCA.statistics()

+----------------------+---------------------------------+
|       Activity       | electricity production, lignite |
|     Method name      |        ('some', 'method')       |
|      Data file       |          lcia_example_2         |
| Unique CFs in method |                3                |
|   Unique CFs used    |                3                |
|  Exc. characterized  |               1210              |
| Exc. uncharacterized |              334757             |
+----------------------+---------------------------------+


We can extend the application of CFs to other exchanges, using the helping functions: `map_aggregate_locations`, `map_disaggregate_location`, `map_dynamic_locations` and `map_remaining_locations_to_global`.

* `map_aggregate_locations` maps exchanges with aggregate locations (e.g., "RER", RLA", etc.) to weighted-average CFs.
* `map_disaggregate_location` maps exchanges with locations that belong to aggregate locations ("CA-QC") to CFs of aggregate locations (e.g., "CA").
* `map_dynamic_locations` maps exchanges with dynamic locations (e.g., "RoW", "RoW") to CFs calculated as the weighted-average of all locations minus those excluded by dynamic locations' perimeter (e.g., "GLO minus CH").
* `map_remaining_locations_to_global` maps exchanges with remaining locations to the global CFs.

In [3]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=("some", "method"),
    filepath="lcia_example_2.json"
)
LCA.lci()

LCA.map_exchanges()

LCA.map_aggregate_locations()
LCA.map_dynamic_locations()
LCA.map_contained_locations()
LCA.map_remaining_locations_to_global()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|██████████| 3/3 [00:00<00:00, 239.62it/s]

Handling static regions...



Processing locations: 100%|██████████| 305/305 [00:00<00:00, 401.76it/s]
Processing locations: 0it [00:00, ?it/s]


...

Handling contained locations...


Processing edges: 100%|██████████| 332506/332506 [00:01<00:00, 258150.22it/s]
Processing edges: 0it [00:00, ?it/s]


Handling remaining exchanges...


Processing remaining global edges: 100%|██████████| 331776/331776 [00:02<00:00, 121699.82it/s]
Processing remaining global edges: 0it [00:00, ?it/s]


3.5288595477886058

In [4]:
LCA.statistics()

+----------------------+--------------------+
|       Activity       |  sawing, softwood  |
|     Method name      | ('some', 'method') |
|      Data file       |   lcia_example_2   |
| Unique CFs in method |         4          |
|   Unique CFs used    |         7          |
|  Exc. characterized  |        6462        |
| Exc. uncharacterized |       329447       |
+----------------------+--------------------+


In [32]:
LCA.weights

{'CH': 8921981, 'FR': 66548530, 'RER': 0}

In [7]:
df = LCA.generate_cf_table()

In [8]:
df

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Carbon dioxide, in air","(natural resource, in air)","rye production, organic","straw, organic",CH,1.015291e-10,1.000000,1.015291e-10
1,"Carbon dioxide, in air","(natural resource, in air)","fruit tree seedling production, for planting","fruit tree seedling, for planting",CH,1.992838e-15,1.000000,1.992838e-15
2,"Carbon dioxide, in air","(natural resource, in air)",beet sugar production,sugar beet pulp,CH,3.855517e-30,1.000000,3.855517e-30
3,"Carbon dioxide, in air","(natural resource, in air)","strawberry production, in unheated greenhouse",strawberry,CH,9.743232e-32,1.000000,9.743232e-32
4,"Carbon dioxide, in air","(natural resource, in air)","potato seed production, Swiss integrated produ...","potato seed, Swiss integrated production, at farm",CH,1.836080e-09,1.000000,1.836080e-09
...,...,...,...,...,...,...,...,...
6457,"Carbon dioxide, fossil","(air, non-urban air or from high stacks)","electricity production, lignite","electricity, high voltage",IN-MP,5.219861e-06,1.881782,9.822640e-06
6458,"Carbon dioxide, fossil","(air,)","petrol production, unleaded, petroleum refiner...","petrol, unleaded",ZA,2.238751e-09,1.881782,4.212841e-09
6459,"Carbon dioxide, from soil or biomass stock","(air, non-urban air or from high stacks)","clear-cutting, grassland to arable land, annua...","land tenure, arable land, measured as carbon n...",BR-BA,8.981452e-10,1.881782,1.690113e-09
6460,"Carbon dioxide, non-fossil","(air, non-urban air or from high stacks)","heat and power co-generation, biogas, gas engine","heat, central or small-scale, other than natur...",MX,4.258051e-12,1.881782,8.012723e-12


We can check the weights used:

In [5]:
LCA.weights

{'CH': 8921981, 'FR': 66548530, 'RER': 0}

By default, if weights are not provided in the consumer sections of LCIA file, population is used. The other option is GDP. To be specified when calling EdgeLICA().

And we can see the weighted-average CFs used:

In [9]:
df.groupby("consumer location")["CF"].mean()

consumer location
AE        1.881782
AR        1.881782
AT        3.000000
AU        1.881782
AU-NSW    1.881782
            ...   
VE        1.881782
VN        1.881782
WEU       1.881782
ZA        1.881782
ZM        1.881782
Name: CF, Length: 227, dtype: float64

We have pre-generated LCIA files for specific methods (AWARE, ImpactWorld+, etc.).
You can get a list like so:

In [37]:
get_available_methods()

[('AWARE 1.2c', 'Country', 'irri', 'apr'),
 ('AWARE 1.2c', 'Country', 'irri', 'aug'),
 ('AWARE 1.2c', 'Country', 'irri', 'dec'),
 ('AWARE 1.2c', 'Country', 'irri', 'feb'),
 ('AWARE 1.2c', 'Country', 'irri', 'jan'),
 ('AWARE 1.2c', 'Country', 'irri', 'jul'),
 ('AWARE 1.2c', 'Country', 'irri', 'jun'),
 ('AWARE 1.2c', 'Country', 'irri', 'mar'),
 ('AWARE 1.2c', 'Country', 'irri', 'may'),
 ('AWARE 1.2c', 'Country', 'irri', 'nov'),
 ('AWARE 1.2c', 'Country', 'irri', 'oct'),
 ('AWARE 1.2c', 'Country', 'irri', 'sep'),
 ('AWARE 1.2c', 'Country', 'irri', 'yearly'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'apr'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'aug'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'dec'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'feb'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jan'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jul'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jun'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'mar'),
 ('AWARE 1.2c', 'Country', 'non', 'irr

Check the other notebooks to see how to use them.